<a href="https://colab.research.google.com/github/MeiChenc/CDX-Tranche-Pricing/blob/main/CDX_Tranche_Pricing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# ================================================================
# 0. Setup: imports & file paths
# ================================================================

import numpy as np
import pandas as pd
import json
from typing import List, Dict
from dataclasses import dataclass
from scipy.optimize import root_scalar, minimize_scalar, minimize
from scipy.stats import norm

np.random.seed(42)  # for reproducibility

# In Colab: upload these two files to the working directory first.
CDX_CONST_FILE = "cdx_constituents_multi_tenor.csv"
CDX_MARKET_FILE = "cdx_market_data_multi_tenor.json"


# ================================================================
# 1. Load data
# ================================================================

# 1.1 Load constituents (we only need number of names & equal weights; recovery set to 40%)
const_df = pd.read_csv(CDX_CONST_FILE)
NUM_NAMES = len(const_df)
weights = np.ones(NUM_NAMES) / NUM_NAMES
RECOVERY_IDX = 0.40   # standard market convention for CDX

print(f"Loaded {NUM_NAMES} constituents.")
print(f"Assumed index-level recovery: {RECOVERY_IDX:.2f}")

# 1.2 Load multi-tenor market data (index + tranches)
with open(CDX_MARKET_FILE, "r") as f:
    mkt_multi = json.load(f)

# Available tenors in years (keys like "1Y","2Y",...)
available_tenors = sorted(mkt_multi.keys(), key=lambda x: float(x.replace("Y", "")))
print("Available tenors in JSON:", available_tenors)

# We focus on these standard tenors:
TENORS_YEARS = [1.0, 2.0, 3.0, 5.0, 7.0, 10.0]

# Extract index "full_index" spreads in bps for each tenor
index_spreads_bps = []
for T in TENORS_YEARS:
    key = f"{int(T)}Y"
    full_index = mkt_multi[key]["full_index"]  # in bps
    index_spreads_bps.append(full_index)

print("Index full_index spreads (bps) per tenor:")
for T, s in zip(TENORS_YEARS, index_spreads_bps):
    print(f"  {T:.0f}Y: {s:.4f} bps")


# ================================================================
# 2. Discount Curve (simple, but standard structure)
# ================================================================

@dataclass
class DiscountCurve:
    """
    Simple discount curve with flat continuously-compounded zero rate.
    For demonstration we assume a flat risk-free rate.
    """
    flat_rate: float  # e.g. 0.03 for 3%

    def df(self, t: float) -> float:
        if t <= 0.0:
            return 1.0
        return np.exp(-self.flat_rate * t)

    def dfs(self, ts: np.ndarray) -> np.ndarray:
        return np.exp(-self.flat_rate * ts)


# For demo: assume flat OIS at 3%
disc_curve = DiscountCurve(flat_rate=0.03)


# ================================================================
# 3. Standard CDS pricing & index hazard bootstrapping
# ================================================================

class CDSBootstrapper:
    """
    Bootstrap piecewise-constant hazard rates from index CDS spreads
    using standard CDS pricing equations:

        PV_prem = S * sum(Δt * DF(t_i) * Q(t_i))
        PV_prot = (1-R) * sum(DF(t_i) * [Q(t_{i-1}) - Q(t_i)])

    where hazard is piecewise-constant by tenor.
    """

    def __init__(self, disc: DiscountCurve, recovery: float = 0.40, freq: int = 4):
        self.disc = disc
        self.R = recovery
        self.freq = freq

    @staticmethod
    def _lambda_for_time(t: float,
                         segment_tenors: List[float],
                         segment_lambdas: List[float]) -> float:
        """
        Given time t, find which tenor segment it belongs to, and return the corresponding lambda.
        Segments are defined by increasing tenor boundaries: [T1, T2, ..., Tn].
        On (0, T1] -> lambda_1, on (T1, T2] -> lambda_2, etc.
        """
        for idx, T in enumerate(segment_tenors):
            if t <= T + 1e-12:
                return segment_lambdas[idx]
        return segment_lambdas[-1]

    def _build_survival_curve(self,
                              coupons: np.ndarray,
                              segment_tenors: List[float],
                              segment_lambdas: List[float]) -> np.ndarray:
        """
        Compute survival Q(t_i) at each coupon time using piecewise-constant hazard rates.
        """
        surv = np.zeros_like(coupons, dtype=float)
        cum_intensity = 0.0
        prev_t = 0.0

        for i, t in enumerate(coupons):
            lam = self._lambda_for_time(t, segment_tenors, segment_lambdas)
            dt = t - prev_t
            cum_intensity += lam * dt
            surv[i] = np.exp(-cum_intensity)
            prev_t = t

        return surv

    def _cds_pv_equation(self,
                         spread_dec: float,
                         T: float,
                         known_tenors: List[float],
                         known_lambdas: List[float],
                         lambda_new: float) -> float:
        """
        Given known hazard segments (for previous tenors), and a trial lambda for the new segment,
        compute PV_prem - PV_prot for target maturity T.
        Root of this function w.r.t lambda_new yields the correct hazard rate for [T_{k-1}, T_k].
        """
        segment_tenors = known_tenors + [T]
        segment_lambdas = known_lambdas + [lambda_new]

        dt = 1.0 / self.freq
        coupons = np.arange(dt, T + 1e-12, dt)

        dfs = self.disc.dfs(coupons)
        surv = self._build_survival_curve(coupons, segment_tenors, segment_lambdas)

        # Premium leg
        pv_prem = np.sum(spread_dec * dt * dfs * surv)

        # Protection leg
        surv_prev = np.concatenate(([1.0], surv[:-1]))
        default_prob = surv_prev - surv
        pv_prot = np.sum((1.0 - self.R) * dfs * default_prob)

        return pv_prem - pv_prot

    def bootstrap_index_hazard(self,
                               tenors: List[float],
                               spreads_bps: List[float]) -> Dict[float, float]:
        """
        Bootstrap piecewise-constant hazard rates from index CDS spreads.

        tenors: [1, 2, 3, 5, 7, 10]
        spreads_bps: index spreads in bps for these tenors.
        return: {T_k: lambda_k} where lambda_k is intensity on (T_{k-1}, T_k].
        """
        tenors = list(tenors)
        spreads_bps = list(spreads_bps)

        known_tenors: List[float] = []
        known_lambdas: List[float] = []
        hazard_by_tenor: Dict[float, float] = {}

        for T, S_bps in zip(tenors, spreads_bps):
            spread_dec = S_bps / 1e4  # bps -> decimal

            def obj(lam_new):
                return self._cds_pv_equation(spread_dec,
                                             T,
                                             known_tenors,
                                             known_lambdas,
                                             lam_new)

            sol = root_scalar(obj, bracket=[1e-6, 10.0], method="brentq")
            lam_star = sol.root
            known_tenors.append(T)
            known_lambdas.append(lam_star)
            hazard_by_tenor[T] = lam_star
            print(f"Bootstrapped lambda for tenor {T}Y: {lam_star:.4f}")

        return hazard_by_tenor


# 3.1 Run hazard bootstrapping
bootstrapper = CDSBootstrapper(disc_curve, recovery=RECOVERY_IDX, freq=4)
hazard_curve = bootstrapper.bootstrap_index_hazard(TENORS_YEARS, index_spreads_bps)

print("\nPiecewise-constant hazard rates (index level):")
for T in sorted(hazard_curve.keys()):
    print(f"  (0, {T}Y] λ = {hazard_curve[T]:.4f}")


# ================================================================
# 4. Survival & portfolio PD per tenor
# ================================================================

def index_survival_at_T(hazard_by_tenor: Dict[float, float], T: float) -> float:
    """
    Compute index survival Q(T) given piecewise-constant hazard segments:
      hazard_by_tenor: {T_k: lambda_k} for segments (T_{k-1}, T_k].
    """
    tenors_sorted = sorted(hazard_by_tenor.keys())
    cum_int = 0.0
    prev = 0.0

    for Tk in tenors_sorted:
        lam = hazard_by_tenor[Tk]
        seg_start = prev
        seg_end = Tk
        if T <= seg_start:
            break
        dt = min(T, seg_end) - seg_start
        if dt > 0:
            cum_int += lam * dt
        prev = seg_end
        if Tk >= T:
            break

    return float(np.exp(-cum_int))


PD_by_tenor: Dict[float, float] = {}
for T in TENORS_YEARS:
    Q_T = index_survival_at_T(hazard_curve, T)
    PD_T = 1.0 - Q_T
    PD_by_tenor[T] = PD_T
    print(f"Tenor {T}Y: Q(T)={Q_T:.4f}, PD(T)={PD_T:.4f}")


# ================================================================
# 5. Gaussian Copula tranche pricer
# ================================================================

class GaussianCopulaTranchePricer:
    """
    One-factor Gaussian copula under LHP approximation for tranche loss at maturity.
    """

    def __init__(self, rho: float, n_points: int = 501):
        self.rho = rho
        self.Y = np.linspace(-5.0, 5.0, n_points)
        pdf = norm.pdf(self.Y)
        dy = self.Y[1] - self.Y[0]
        self.weights = pdf * dy / np.sum(pdf * dy)

    def conditional_pd(self, PD_T: float, y: float) -> float:
        PD_T = np.clip(PD_T, 1e-8, 1.0 - 1e-8)
        K = norm.ppf(PD_T)
        num = K - np.sqrt(self.rho) * y
        den = np.sqrt(1.0 - self.rho)
        return float(norm.cdf(num / den))

    def tranche_el(self,
                   PD_T: float,
                   attach: float,
                   detach: float,
                   recovery: float = 0.40) -> float:
        """
        Expected tranche loss as fraction of tranche notional.
        """
        lgd = 1.0 - recovery
        width = detach - attach

        losses = np.zeros_like(self.Y, dtype=float)
        for i, y in enumerate(self.Y):
            cond_pd = self.conditional_pd(PD_T, y)
            L = lgd * cond_pd   # LHP approx
            losses[i] = np.clip(L - attach, 0.0, width) / width

        EL = float(np.sum(losses * self.weights))
        return EL


def fair_tranche_spread(discount_curve: DiscountCurve,
                        maturity: float,
                        EL_T: float,
                        recovery: float = 0.40,
                        freq: int = 4) -> float:
    """
    Approximate fair running spread for a tranche given expected loss at maturity.
    This is a standard approximation:
      - premium leg uses average outstanding tranche notional
      - protection leg approximated by EL_T discounted to T.
    """
    dt = 1.0 / freq
    payment_times = np.arange(dt, maturity + 1e-12, dt)
    dfs = discount_curve.dfs(payment_times)

    # Approximate average outstanding tranche notional
    avg_notional = 1.0 - 0.5 * EL_T
    risky_pv01 = float(np.sum(dfs * dt * avg_notional))

    df_T = discount_curve.df(maturity)
    pv_prot = EL_T * df_T

    s = pv_prot / risky_pv01  # decimal (e.g. 0.02 = 200 bps)
    return float(s)


def price_tranches_gaussian(PD_T: float,
                            rho: float,
                            disc_curve: DiscountCurve,
                            maturity: float,
                            recovery: float = 0.40) -> Dict[str, Dict[str, float]]:
    copula = GaussianCopulaTranchePricer(rho)
    tr_defs = [
        ("equity_0_3", 0.00, 0.03),
        ("mezz_3_7",   0.03, 0.07),
        ("mezz_7_10",  0.07, 0.10),
        ("senior_10_15", 0.10, 0.15),
        ("senior_15_100", 0.15, 1.00),
    ]
    results = {}
    for name, A, B in tr_defs:
        EL = copula.tranche_el(PD_T, A, B, recovery)
        s = fair_tranche_spread(disc_curve, maturity, EL, recovery)
        results[name] = {"EL": EL, "spread": s}
    return results


# ================================================================
# 6. Variance-Gamma sampler + GVG Copula pricer
# ================================================================

def sample_variance_gamma(n: int, theta: float = 0.0, sigma: float = 1.0, nu: float = 1.0) -> np.ndarray:
    """
    Sample n Variance-Gamma distributed values using standard construction:
      G ~ Gamma(nu/2, scale=2/nu)
      Z ~ N(0,1)
      Y = theta * G + sigma * sqrt(G) * Z
    """
    G = np.random.gamma(shape=nu / 2.0, scale=2.0 / nu, size=n)
    Z = np.random.normal(size=n)
    Y = theta * G + sigma * np.sqrt(G) * Z
    return Y


class GVGCopulaTranchePricer:
    """
    Mixed Gaussian + Variance-Gamma one-factor copula:
      - with prob p_high: VG factor with rho_high (high-corr, fat-tail regime)
      - with prob (1 - p_high): Gaussian factor with rho_low (low-corr regime)
    """

    def __init__(self,
                 rho_high: float,
                 rho_low: float,
                 p_high: float,
                 theta: float = 0.0,
                 sigma: float = 1.0,
                 nu: float = 1.0,
                 n_points: int = 2000):
        self.rho_high = rho_high
        self.rho_low = rho_low
        self.p_high = p_high
        self.theta = theta
        self.sigma = sigma
        self.nu = nu

        n_high = int(n_points * p_high)
        n_low = n_points - n_high

        # High-correlation VG regime
        Y_high = sample_variance_gamma(
            n=n_high,
            theta=self.theta,
            sigma=self.sigma,
            nu=self.nu
        )

        # Low-correlation Gaussian regime
        Y_low = np.random.normal(size=n_low)

        self.Y = np.concatenate([Y_high, Y_low])
        self.is_high = np.concatenate([
            np.ones(n_high, dtype=bool),
            np.zeros(n_low, dtype=bool),
        ])
        self.weights = np.ones_like(self.Y, dtype=float) / len(self.Y)

    def conditional_pd(self, PD_T: float, idx: int) -> float:
        PD_T = np.clip(PD_T, 1e-8, 1.0 - 1e-8)
        K = norm.ppf(PD_T)
        y = self.Y[idx]
        rho = self.rho_high if self.is_high[idx] else self.rho_low
        num = K - np.sqrt(rho) * y
        den = np.sqrt(1.0 - rho)
        return float(norm.cdf(num / den))

    def tranche_el(self,
                   PD_T: float,
                   attach: float,
                   detach: float,
                   recovery: float = 0.40) -> float:
        lgd = 1.0 - recovery
        width = detach - attach

        losses = np.zeros_like(self.Y, dtype=float)
        for i in range(len(self.Y)):
            cond_pd = self.conditional_pd(PD_T, i)
            L = lgd * cond_pd
            losses[i] = np.clip(L - attach, 0.0, width) / width

        EL = float(np.sum(losses * self.weights))
        return EL


def price_tranches_gvg(PD_T: float,
                       params: Dict[str, float],
                       disc_curve: DiscountCurve,
                       maturity: float,
                       recovery: float = 0.40) -> Dict[str, Dict[str, float]]:
    pricer = GVGCopulaTranchePricer(
        rho_high=params["rho_high"],
        rho_low=params["rho_low"],
        p_high=params["p_high"],
        theta=params.get("theta", 0.0),
        sigma=params.get("sigma", 1.0),
        nu=params.get("nu", 1.0),
        n_points=2000,
    )
    tr_defs = [
        ("equity_0_3", 0.00, 0.03),
        ("mezz_3_7",   0.03, 0.07),
        ("mezz_7_10",  0.07, 0.10),
        ("senior_10_15", 0.10, 0.15),
        ("senior_15_100", 0.15, 1.00),
    ]
    results = {}
    for name, A, B in tr_defs:
        EL = pricer.tranche_el(PD_T, A, B, recovery)
        s = fair_tranche_spread(disc_curve, maturity, EL, recovery)
        results[name] = {"EL": EL, "spread": s}
    return results


# ================================================================
# 7. Calibration & pricing error per tenor
# ================================================================

def get_market_tranche_quotes_for_tenor(tenor_years: float) -> Dict[str, float]:
    """
    Extract market tranche quotes for a given tenor T from the JSON structure.
    Returns:
      {
        'equity_0_3_upfront': float (%),
        'equity_0_3_running': float (bps),
        'mezz_3_7': float (bps),
        'mezz_7_10': float (bps),
        'senior_10_15': float (bps),
        'senior_15_100': float (bps)
      }
    """
    key = f"{int(tenor_years)}Y"
    d = mkt_multi[key]
    out = {
        "equity_0_3_upfront": d["equity_0_3_upfront"],
        "equity_0_3_running": d["equity_0_3_running"],
        "mezz_3_7": d["mezz_3_7"],
        "mezz_7_10": d["mezz_7_10"],
        "senior_10_15": d["senior_10_15"],
        "senior_15_100": d["senior_15_100"],
    }
    return out


def calibrate_gaussian_for_tenor(PD_T: float,
                                 disc_curve: DiscountCurve,
                                 maturity: float,
                                 mkt_quotes: Dict[str, float],
                                 recovery: float = 0.40) -> float:
    """
    Calibrate single rho_T to match equity 0-3% upfront (simplified).
    We approximate equity upfront as EL * 100 (%).
    """
    target_upfront = mkt_quotes["equity_0_3_upfront"]

    def obj(rho):
        if rho <= 0.0 or rho >= 1.0:
            return 1e6
        copula = GaussianCopulaTranchePricer(rho)
        EL_eq = copula.tranche_el(PD_T, 0.0, 0.03, recovery)
        model_upfront = EL_eq * 100.0
        return (model_upfront - target_upfront) ** 2

    res = minimize_scalar(obj, bounds=(0.01, 0.99), method="bounded")
    return float(res.x)


def calibrate_gvg_for_tenor(PD_T: float,
                            disc_curve: DiscountCurve,
                            maturity: float,
                            mkt_quotes: Dict[str, float],
                            recovery: float = 0.40) -> Dict[str, float]:
    """
    Calibrate GVG parameters (rho_high, rho_low, p_high).
    VG tail parameters (theta, sigma, nu) fixed for now.

    Objective: sum of squared errors across equity upfront and all mezz/senior spreads.
    """
    target_upfront = mkt_quotes["equity_0_3_upfront"]

    def obj(theta_vec):
        rho_high, rho_low, p_high = theta_vec
        if not (0.0 < rho_low < 1.0 and 0.0 < rho_high < 1.0 and 0.0 < p_high < 1.0):
            return 1e6

        params = {
            "rho_high": float(rho_high),
            "rho_low": float(rho_low),
            "p_high": float(p_high),
            "theta": 0.0,
            "sigma": 1.0,
            "nu": 1.0,
        }
        model_prices = price_tranches_gvg(PD_T, params, disc_curve, maturity, recovery)

        # Equity upfront (approx)
        EL_eq = model_prices["equity_0_3"]["EL"]
        model_upfront = EL_eq * 100.0
        err_eq = (model_upfront - target_upfront) ** 2

        # Other tranches: compare running spread (bps)
        err_others = 0.0
        for name in ["mezz_3_7", "mezz_7_10", "senior_10_15", "senior_15_100"]:
            s_model_bps = model_prices[name]["spread"] * 1e4
            s_mkt_bps = mkt_quotes[name]
            err_others += (s_model_bps - s_mkt_bps) ** 2

        return err_eq + err_others

    x0 = np.array([0.5, 0.2, 0.5])
    bounds = [(0.05, 0.95), (0.01, 0.95), (0.05, 0.95)]

    res = minimize(obj, x0=x0, bounds=bounds, method="L-BFGS-B")
    rho_high, rho_low, p_high = res.x

    params = {
        "rho_high": float(rho_high),
        "rho_low": float(rho_low),
        "p_high": float(p_high),
        "theta": 0.0,
        "sigma": 1.0,
        "nu": 1.0,
    }
    return params


def compute_pricing_errors_for_tenor(T: float,
                                     PD_T: float,
                                     disc_curve: DiscountCurve,
                                     recovery: float = 0.40) -> Dict[str, object]:
    """
    For a given tenor T:
      - calibrate Gaussian rho_T
      - calibrate GVG parameters
      - price all tranches under both models
      - compute pricing errors vs market quotes
    Returns:
      dict with DataFrames for Gaussian and GVG comparison tables and MAE metrics.
    """
    mkt_quotes = get_market_tranche_quotes_for_tenor(T)

    # 1) Gaussian calibration & pricing
    rho_T = calibrate_gaussian_for_tenor(PD_T, disc_curve, T, mkt_quotes, recovery)
    gauss_prices = price_tranches_gaussian(PD_T, rho_T, disc_curve, T, recovery)

    # 2) GVG calibration & pricing
    gvg_params = calibrate_gvg_for_tenor(PD_T, disc_curve, T, mkt_quotes, recovery)
    gvg_prices = price_tranches_gvg(PD_T, gvg_params, disc_curve, T, recovery)

    rows_gauss = []
    rows_gvg = []

    for name in ["equity_0_3", "mezz_3_7", "mezz_7_10", "senior_10_15", "senior_15_100"]:
        if name == "equity_0_3":
            # Upfront comparison (%)
            mkt_val = mkt_quotes["equity_0_3_upfront"]
            model_gauss = gauss_prices[name]["EL"] * 100.0
            model_gvg = gvg_prices[name]["EL"] * 100.0
            unit = "%"
        else:
            # Running spread comparison (bps)
            mkt_val = mkt_quotes[name]
            model_gauss = gauss_prices[name]["spread"] * 1e4
            model_gvg = gvg_prices[name]["spread"] * 1e4
            unit = "bps"

        rows_gauss.append({
            "Tranche": name,
            "Market": mkt_val,
            "Model": model_gauss,
            "Error": model_gauss - mkt_val,
            "Unit": unit,
        })
        rows_gvg.append({
            "Tranche": name,
            "Market": mkt_val,
            "Model": model_gvg,
            "Error": model_gvg - mkt_val,
            "Unit": unit,
        })

    df_gauss = pd.DataFrame(rows_gauss)
    df_gvg = pd.DataFrame(rows_gvg)

    mae_gauss = df_gauss["Error"].abs().mean()
    mae_gvg = df_gvg["Error"].abs().mean()

    print(f"\n=== Tenor {T}Y ===")
    print(f"Calibrated Gaussian rho_T: {rho_T:.4f}")
    print(f"Calibrated GVG params: {gvg_params}")
    print("\nGaussian pricing vs market:")
    print(df_gauss.to_string(index=False))
    print(f"Gaussian MAE (mix units): {mae_gauss:.4f}")

    print("\nGVG pricing vs market:")
    print(df_gvg.to_string(index=False))
    print(f"GVG MAE (mix units): {mae_gvg:.4f}")

    return {
        "gaussian": df_gauss,
        "gvg": df_gvg,
        "rho_gauss": rho_T,
        "params_gvg": gvg_params,
        "mae_gauss": mae_gauss,
        "mae_gvg": mae_gvg,
    }


# ================================================================
# 8. Run experiment across all tenors
# ================================================================

results_by_tenor = {}

for T in TENORS_YEARS:
    PD_T = PD_by_tenor[T]
    res_T = compute_pricing_errors_for_tenor(T, PD_T, disc_curve, RECOVERY_IDX)
    results_by_tenor[T] = res_T

print("\nDone. 'results_by_tenor' now holds detailed comparison tables and parameters for each tenor.")


Loaded 125 constituents.
Assumed index-level recovery: 0.40
Available tenors in JSON: ['1Y', '2Y', '3Y', '5Y', '7Y', '10Y']
Index full_index spreads (bps) per tenor:
  1Y: 22.3900 bps
  2Y: 30.0200 bps
  3Y: 38.5900 bps
  5Y: 51.9700 bps
  7Y: 77.9900 bps
  10Y: 95.5500 bps
Bootstrapped lambda for tenor 1.0Y: 0.0037
Bootstrapped lambda for tenor 2.0Y: 0.0063
Bootstrapped lambda for tenor 3.0Y: 0.0094
Bootstrapped lambda for tenor 5.0Y: 0.0123
Bootstrapped lambda for tenor 7.0Y: 0.0256
Bootstrapped lambda for tenor 10.0Y: 0.0247

Piecewise-constant hazard rates (index level):
  (0, 1.0Y] λ = 0.0037
  (0, 2.0Y] λ = 0.0063
  (0, 3.0Y] λ = 0.0094
  (0, 5.0Y] λ = 0.0123
  (0, 7.0Y] λ = 0.0256
  (0, 10.0Y] λ = 0.0247
Tenor 1.0Y: Q(T)=0.9963, PD(T)=0.0037
Tenor 2.0Y: Q(T)=0.9900, PD(T)=0.0100
Tenor 3.0Y: Q(T)=0.9807, PD(T)=0.0193
Tenor 5.0Y: Q(T)=0.9568, PD(T)=0.0432
Tenor 7.0Y: Q(T)=0.9090, PD(T)=0.0910
Tenor 10.0Y: Q(T)=0.8442, PD(T)=0.1558

=== Tenor 1.0Y ===
Calibrated Gaussian rho_T: 0.5